In [1]:
import csv
import re

def generate_insert_statements(
    csv_file, 
    create_stream_statement, 
    output_file, 
    table_name,
    encoding='utf-8'
):
    # Extract content within parentheses
    stream_content = re.search(r'\((.*?)\)', create_stream_statement, re.DOTALL).group(1)
    
    # Split content into individual column definitions
    columns = re.findall(r'([a-zA-Z_]+)\s+([a-zA-Z_]+)', stream_content)

    # Extract column names and data types, stripping BOM if present
    column_names_create_stream = [col[0].strip() for col in columns]
    data_types_create_stream = {col[0].strip(): col[1] for col in columns}

    # Open the CSV file
    with open(csv_file, newline='', encoding=encoding, errors='ignore') as csvfile:
        reader = csv.DictReader(csvfile)
        insert_statements = []

        # Iterate over each row in the CSV
        for row in reader:
            columns = []
            values = []

            # Iterate over each column in the row
            for field, value in row.items():
                if field in column_names_create_stream:
                    # Column name exists in create_stream_statement
                    data_type = data_types_create_stream[field]
                    if value is None:
                        columns.append(field)
                        values.append("")
                    else:
                        if data_type == "DOUBLE" or data_type == "INTEGER":
                            value_str = str(value)
                            if value_str.strip() == "":
                                columns.append(field)
                                values.append("")
                            else:
                                columns.append(field)
                                values.append(value_str)
                        elif data_type == "TIMESTAMP" or data_type == "DATE" or data_type == "TIME":
                            columns.append(field)
                            values.append("'" + str(value) + "'")
                        elif data_type == "VARCHAR" or data_type == "TEXT":
                            columns.append(field)
                            values.append("'" + value.replace("'", "''") + "'")
                        # Handle other data types as needed

            # Construct the INSERT INTO statement with the specified table name
            insert_statement = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({', '.join(values)});"
            insert_statements.append(insert_statement)

    # Write insert statements to output file
    with open(output_file, 'w', encoding=encoding, errors='ignore') as f:
        for statement in insert_statements:
            f.write(statement + '\n')

In [2]:
create_stream_statement = """
CREATE STREAM household_energy_data (
    energy_type VARCHAR,
    energy_date VARCHAR,
    start_time VARCHAR,
    end_time VARCHAR,
    energy_usage INTEGER,
    energy_units VARCHAR,
    energy_cost VARCHAR,
    notes VARCHAR
    )
"""

input_data_file = '/Users/briandunn/Desktop/ksqldb projects/Household Energy Data/data/D202.csv'

output_file_location = '/Users/briandunn/Desktop/ksqldb projects/Household Energy Data/statements/producers/household_energy_inserts.sql'

generate_insert_statements(
    input_data_file, 
    create_stream_statement, 
    output_file_location,
    "household_energy_data"
    )